In [1]:
import django
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
import parliament.models as pm

from django.db.models import Count, Sum
import random
p = Project.objects.get(pk=134)
p

<Project: Coal-discourse>

In [3]:
search = pm.Search.objects.get(id=75)

uts = search.utterance_set.all()

uts.count()

5559

In [4]:
# create query and tag

q, created = Query.objects.get_or_create(
    title="{} (psearch id: {})".format(search.title, search.id),
    text=search.text,
    project=p,
    creator=User.objects.get(username="muef")
)


users = User.objects.filter(username__in=["muef"])
print(users)

for u in users:
    q.users.add(u)

tag, created = Tag.objects.get_or_create(
    query=q,
    title="Kohle pdf old",
    text="kohle",
    document_linked=False,
    utterance_linked=True
)

<QuerySet [<User: muef>]>


In [39]:
# create DocOwnership objects for review

sample_size = 50

ut_ids = list(uts.values_list('pk',flat=True))
ut_sample = uts.filter(id__in=random.sample(ut_ids,sample_size*users.count()))

for i,ut in enumerate(ut_sample):
    u = users[i%users.count()]
    #do = DocOwnership.objects.filter(tag=tag)[i]
    #if do.relevant==0:
    #    do.utterance = ut
    #    do.save()
    do, created = DocOwnership.objects.get_or_create(
        utterance=ut,
        document_linked=False,
        utterance_linked=True,
        user=u,
        query=q,
        tag=tag
    )

In [40]:
DocOwnership.objects.filter(tag=tag).count()

50

In [7]:
# delete unrated docownership objects
#DocOwnership.objects.filter(tag=tag, relevant=0).delete()

(24, {'scoping.DocOwnership': 24})

In [5]:
# Analyze note objects

qs = Query.objects.filter(
    project=p,
    creator=User.objects.get(username="muef")
)

print(qs)

tags = Tag.objects.filter(query=qs[0])

print(tags)

<QuerySet [<Query: Kohle pdf 01/01 - 13/85 speech (psearch id: 75)>, <Query: Kohle pdf test>, <Query: Kohle pdf old>, <Query: Kohle pdf utterance all (search id 39)>]>
<QuerySet [<Tag: Kohle pdf old>]>


In [6]:
notes = Note.objects.filter(tag=tags[0]).values('text', 'utterance__document__date',
                                                'utterance__document__parlperiod__n',
                                                'utterance__document__sitting', 'utterance__id').order_by('utterance__document__date')

notes_df = pd.DataFrame.from_dict(notes)

notes_df["text"] = notes_df["text"].str.strip()

notes_df #[notes_df["text"].str.contains("wrong speaker")]

,text,utterance__document__date,utterance__document__parlperiod__n,utterance__document__sitting,utterance__id
0,wrong speaker,1952-01-09,1,182,2735250
1,procedural,1959-01-29,3,59,2678761
2,question,1963-02-13,4,60,2643868
3,question,1966-09-21,5,57,2597062
4,wrong speaker,1968-11-13,5,194,2624226
5,end not correct,1969-02-26,5,218,2628645
6,question,1971-11-04,6,148,2577070
7,end not correct,1971-11-04,6,148,2577070
8,question,1975-12-11,7,208,2539959
9,question,1977-11-24,8,58,2457913


In [5]:
notes_df.to_csv('assign_parl_review_notes.csv')

In [17]:
notes_df[notes_df["text"].str.contains("end")]

,text,utterance__document__date,utterance__document__parlperiod__n,utterance__document__sitting
9,end of speech not identified correctly,1968-06-25,5,182
10,end not correct,1971-11-11,6,151
17,end of speech not correct,1989-03-08,11,130
18,end not correct,1991-11-06,12,53


In [22]:
# second round of notes

tags = Tag.objects.filter(query=qs[2])

print(tags)

notes = Note.objects.filter(tag=tags[0]).values('text', 'utterance__document__date',
                                                'utterance__document__parlperiod__n',
                                                'utterance__document__sitting', 'utterance__id').order_by('utterance__document__date')

notes_df = pd.DataFrame.from_dict(notes)

notes_df["text"] = notes_df["text"].str.strip()

notes_df #[notes_df["text"].str.contains("wrong speaker")]

<QuerySet [<Tag: Kohle pdf old>]>


,text,utterance__document__date,utterance__document__parlperiod__n,utterance__document__sitting,utterance__id
0,wrong speaker,1952-01-10,1,183,2735311
1,procedural,1954-04-08,2,24,2696227
2,wrong speaker,1956-11-29,2,174,2712677
3,procedural,1957-01-11,2,183,2713370
4,procedural,1958-04-16,3,22,2675645
5,wrong speaker,1959-11-06,3,88,2681640
6,question,1959-12-03,3,92,2682322
7,question,1961-06-29,3,164,2693744
8,question,1962-05-16,4,30,2639096
9,end of speech not identified correctly,1968-06-25,5,182,2622180


In [26]:
# delete query
#Query.objects.get(id=6803).delete()

(53,
 {'scoping.DocOwnership': 50,
  'scoping.DocPar_tag': 0,
  'scoping.DocRel': 0,
  'scoping.Doc_tag': 0,
  'scoping.Note': 0,
  'scoping.Query': 1,
  'scoping.Query_queries': 0,
  'scoping.Query_users': 1,
  'scoping.Tag': 1,
  'scoping.UserTag': 0})

### reviewing text

In [32]:
ut = pm.Paragraph.objects.filter(utterance__id=2146830)
for para in ut:
    print(para.text)

selbstverständlich aus meiner und aus unserer Sicht vorhanden, Kollege Reuschenbach. Meine Damen und Herren, ich darf mich jetzt noch einmal dem Thema zuwenden und sagen, daß wir von dieser Stelle hier aus schon sehr viele Stahldebatten geführt haben. Dabei haben wir immer wieder zum Ausdruck gebracht: Gegen die Macht der Staatskassen einzelner EG-Staaten, die ihre Stahlindustrie rechtswidrig subventionieren, kommt die deutsche Stahlindustrie nicht an. Sie kann dabei nicht mithalten. Wenn es stimmt, daß Italien und Spanien ihren Stahlfirmen vertragswidrig mit Milliardensubventionen unter die Arme greifen und damit die Wettbewerbsordnung sowie das geltende EG-Recht vor die Hunde gehen, dann zeigt das meiner Meinung nach, daß es zuwenig ist, wenn die EG ein umständliches Prüfungsverfahren in Gang setzt. Bis da die Ergebnisse vorliegen – und wir wissen ja leider, wie in Brüssel gearbeitet wird –, ist die Zeit für die deutschen Unternehmen zum Vorteil anderer davongelaufen. Wie man hört, s